In [1]:
##### If you have full datafile(2014_2024.pkl), you can start from here #####
import pandas as pd

df = pd.read_pickle('./seoul_real_estate/2014_2024.pkl')

In [ ]:
# split '시군구' to '행정구', '행정동' columns 시 <- every data is "서울특별시", so ignore it
df['행정구'] = df['시군구'].str.split(' ').str[1].astype('category')
df['행정동'] = df['시군구'].str.split(' ').str[2].astype('category')

# split '도로명' to '도로명', '도로번호' columns. '도로번호' type is int, '도로명' type is category
df['도로번호'] = df['도로명'].str.split(' ').str[-1].astype('category')
df['도로명'] = df['도로명'].str.split(' ').str[:-1].str.join('').astype('category') # remove last element and join(in case of exceptional situation)

df['계약일'] = (df['계약년월'].astype(str) + df['계약일'].astype(str).str.zfill(2)).astype('datetime64[ns]')
# Columns to drop = ['계약년월', '시군구', '주택유형', '해제사유발생일', '매수자', '매도자', '중개사소재지', '등기일자', '계약기간', '갱신요구권 사용', '종전계약 보증금(만원)', '종전계약 월세(만원)', '계약구분', '번지']
df = df.drop(columns=['계약년월', '시군구', '주택유형', '해제사유발생일', '매수자', '매도자', '중개사소재지', '등기일자', '계약기간', '갱신요구권 사용', '종전계약 보증금(만원)', '종전계약 월세(만원)', '계약구분', '번지'])

# reordering columns for better readability -> 행정구 행정동 본번 부번 도로명 단지명 건축년도 동 층 전용면적(㎡) 계약일 계약형태 거래유형 거래금액(만원) 보증금(만원)	월세금(만원)
df = df[['행정구', '행정동', '본번', '부번', '도로명', '도로번호', '단지명', '건축년도', '동', '층', '전용면적(㎡)', '계약일', '계약형태', '거래유형', '거래금액(만원)', '보증금(만원)', '월세금(만원)']]

display(df.head(5))
display(df.info(memory_usage='deep'))

In [ ]:
df.to_pickle('./seoul_real_estate/2014_2024_cleaned.pkl')

In [ ]:
##### If you have clean datafile(2014_2024.pkl), you can start from here #####
import pandas as pd

df = pd.read_pickle('./seoul_real_estate/2014_2024_cleaned.pkl')

In [2]:
from sqlalchemy import create_engine
from urllib import parse

# get connection information from .env file
with open('.env', 'r') as f:
  conn_info = dict(
    tuple(line.strip().split('='))
    for line in f
  )
# create engine for mysql
engine = create_engine((f"mysql://{conn_info['USER']}:{parse.quote(conn_info['PASSWORD'])}@{conn_info['HOST']}:{conn_info['PORT']}/{conn_info['DATABASE']}"))

In [1]:
# insert df to mysql as table
df.to_sql('seoul_real_estate', con=engine, if_exists='replace', index=False)

# check if data is inserted
sql = "SELECT * FROM seoul_real_estate LIMIT 5;"
df = pd.read_sql(sql, engine)

cur = engine.connect()
cur.execute(sql)

# clear engine queue
engine.dispose()

NameError: name 'df' is not defined

In [ ]:
import os

# get file list in directory ./seoul_real_estate/indicator_data_per_region
file_list = os.listdir('./seoul_real_estate/Indicator_data_per_region')

for f in file_list:
  # read file
  df = pd.read_csv(f'./seoul_real_estate/Indicator_data_per_region/{f}', encoding='utf-8')
  # rename Unnamed: 0 to 연도
  df = df.rename(columns={'Unnamed: 0': '연도'})
  # from column 1 to last column replace column name to {column_name}.split('_')[1]
  df.columns = ['연도'] + [f'{col.split("_")[1]}' for col in df.columns[1:]]
  # make columns "행정구"
  df['행정구'] = f.split('_')[1]

  # insert df to mysql as table "Indicator_data_per_region", if table already exists, append data, if not, create table and use "행정구", "연도" as primary key
  df.to_sql('Indicator_data_per_region', con=engine, if_exists='append', index=False)


In [ ]:
df = pd.read_sql(sql, con=engine)

display(df.head(5))

## 컬럼 분석
- 행정구 : 행정구를 나타내는 컬럼
- 행정동 : 행정동을 나타내는 컬럼
- 본번 : 번지 중 앞 번호
- 부번 : 번지 중 뒷 번호(존재하지 않는 경우 0)
- 도로명 : 도로명주소 중 도로명
- 도로번호 : 도로명주소 중 도로번호
- 단지명 : 아파트 단지명
- 건축년도 : 아파트 건축년도
- 동 : 아파트 동 번호
- 층 : 아파트 층수
- 전용면적(㎡) : 아파트 전용면적 (평방미터)
- 계약형태 : 전세, 매매, 월세
- 거래유형 : 아파트 거래 유형(중개, 직거래) <- 직거래일 시 특수관계인간 거래일 수 있음
- 거래금액(만원) : 매매일 경우 거래금액(만원)
- 보증금(만원) : 전월세일 경우 보증금(만원)
- 월세금(만원) : 월세일 경우 월세금(만원)

In [2]:
##### If you have clean datafile(2014_2024.pkl), you can start from here #####
import pandas as pd

df = pd.read_pickle('./seoul_real_estate/2014_2024_cleaned.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2775194 entries, 0 to 92081
Data columns (total 17 columns):
 #   Column    Dtype         
---  ------    -----         
 0   행정구       category      
 1   행정동       category      
 2   본번        int64         
 3   부번        int64         
 4   도로명       category      
 5   도로번호      category      
 6   단지명       category      
 7   건축년도      int64         
 8   동         category      
 9   층         int64         
 10  전용면적(㎡)   float64       
 11  계약일       datetime64[ns]
 12  계약형태      category      
 13  거래유형      category      
 14  거래금액(만원)  int64         
 15  보증금(만원)   int64         
 16  월세금(만원)   int64         
dtypes: category(8), datetime64[ns](1), float64(1), int64(7)
memory usage: 246.3 MB


In [3]:
# get unique set of "행정구" "도로명" "도로번호"
unique_set = df[['행정구', '도로명', '도로번호']].drop_duplicates()
display(unique_set.head(5))
display(len(unique_set))

,행정구,도로명,도로번호
0,종로구,필운대로,69
1,종로구,사직로8길,4
31,종로구,경희궁길,57
46,종로구,새문안로3길,12
48,종로구,율곡로20길,24


10481

In [4]:
import time # for sleep
import tqdm # for progress bar

for set in tqdm.tqdm(unique_set.values):
  try:
    # get data from df where "행정구" "도로명" "도로번호" is same as set. and "계약형태" is "매매"
    data = df[(df['행정구'] == set[0]) & (df['도로명'] == set[1]) & (df['도로번호'] == set[2]) & (df['계약형태'] == '매매')]
    # get data from df where "행정구" "도로명" "도로번호" is same as set. and "계약형태" is "전세"
    data2 = df[(df['행정구'] == set[0]) & (df['도로명'] == set[1]) & (df['도로번호'] == set[2]) & (df['계약형태'] == '전세')]


    # drop [동 층 계약형태 거래금액(만원) 보증금(만원) 월세금(만원) 거래유형 계약일 전용면적] columns
    t_df = data[:1].drop(columns=['동', '층', '계약형태', '거래금액(만원)', '보증금(만원)', '월세금(만원)', '거래유형', '계약일', '전용면적(㎡)'])
    # group by "계약일" monthly and group by "전용면적(㎡)"
    data = data.groupby([data['계약일'].dt.to_period('M'), data['전용면적(㎡)']])
    data = data['거래금액(만원)'].mean().unstack().melt(ignore_index=False).reset_index().dropna()

    data2 = data2.groupby([data2['계약일'].dt.to_period('M'), data2['전용면적(㎡)']])
    data2 = data2['보증금(만원)'].mean().unstack().melt(ignore_index=False).reset_index().dropna()
    # rename columns
    data.columns = ['계약일', '전용면적(㎡)', '거래금액(만원)']
    data2.columns = ['계약일', '전용면적(㎡)', '전세보증금(만원)']

    # join 2 dataframes
    data = data.merge(data2, on=['계약일', '전용면적(㎡)'], how='outer')
    # add t_df to data every row and order by "계약일", "전용면적(㎡)"
    cols_to_fill = t_df.columns
    data = pd.concat([t_df, data], axis=1)
    data.loc[:, cols_to_fill] = data.loc[:, cols_to_fill].ffill()
    data.sort_values(by=['계약일', '전용면적(㎡)'])
    # rename columns to gugun dong main_bunge sub_bunge road_name road_num danji_name build_year deal_month size avg_price
    data.columns = ['gugun', 'dong', 'main_bunge', 'sub_bunge', 'road_name', 'road_num', 'danji_name', 'build_year', 'deal_month', 'size', 'avg_sale_price', 'avg_rent_deposit_price']
    
    # insert data to mysql as table "seoul_real_estate_price_per_month"
    data.to_sql('seoul_real_estate_price_per_month', con=engine, if_exists='append', index=False)
  except:
    print(f'Error occured in {set}', end=' ')
    continue

  time.sleep(0.02) # sleep for 0.03 sec to prevent server overload

  0%|          | 0/10481 [00:00<?, ?it/s]

 57%|█████▋    | 5995/10481 [08:46<05:41, 13.14it/s]

Error occured in ['종로구' '삼일대로' '428']
Error occured in ['종로구' '성균관로17길' '35']


 57%|█████▋    | 6017/10481 [08:48<05:43, 13.00it/s]

Error occured in ['종로구' '종로56길' '42-8']


...

100%|█████████▉| 10473/10481 [14:36<00:00, 16.69it/s]

Error occured in ['서초구' '방배로6길' '10-3']
Error occured in ['서초구' '주흥1길' '19']
Error occured in ['서초구' '반포대로18길' '13-3']


100%|█████████▉| 10479/10481 [14:36<00:00, 16.47it/s]

Error occured in ['강남구' '언주로' '624']
Error occured in ['송파구' '성내천로6길' '1']
Error occured in ['강동구' '고덕로98길' '75']
Error occured in ['강동구' '천중로40길' '72']


100%|██████████| 10481/10481 [14:36<00:00, 11.95it/s]
